In [38]:
import pandas as pd

path = "/Users/anamargarida/Local/Coding/project/data_clean/full_text.parquet"
df = pd.read_parquet(path)


allowed = ["recommendations", "changes_seen"]

df = df[df["text_type"].isin(allowed)].copy()

df.shape
df.head()

,id,district,community,participant_profile,text_type,text,source
2641,602864599,Moatize,Bagamoio,Adolescent girl,changes_seen,purchase of bricks,q1
2642,602871219,Moatize,Bagamoio,Adolescent girl,changes_seen,still,q1
2643,602878149,Moatize,Bagamoio,Adolescent girl,changes_seen,no,q1
2644,602888552,Moatize,Bagamoio,Adolescent girl,changes_seen,towing at homes,q1
2645,602896835,Moatize,Bagamoio,Adolescent girl,changes_seen,i dont know,q1


In [39]:
df.columns

Index(['id', 'district', 'community', 'participant_profile', 'text_type',
       'text', 'source'],
      dtype='object')

In [40]:
df["very_short"] = df["text"].str.len() < 10
df["very_short"].mean()

np.float64(0.034518289541473464)

In [41]:
df["text"].isna().mean()

np.float64(0.0)

In [42]:
df["text"].str.len().describe()

count    1941.000000
mean       55.167955
std        39.178005
min         2.000000
25%        25.000000
50%        47.000000
75%        75.000000
max       326.000000
Name: text, dtype: float64

In [43]:
df["text"].sample(20).tolist()

['the change is that many young women have returned to school',
 'i dont see any change because i never participated',
 'there was a decrease in premature marriages',
 'decrease in premature marriage and increase in safe births but it was thanks to the information they transmitted to us during the period they provided the service',
 'i dont know',
 'certainty in girls',
 'continuing with studies',
 'i recommend that all girls take care of their health',
 'now i know that sex only after im 18 years old or if i feel ready to start i should seek health care to get tested to see if my partner or i are in good health or not',
 'more insight into the subject',
 'thank you very much for your interest in teaching us and knowing about our rights and how to avoid premature engravings',
 'now girls behavior',
 'i would like to thank the tsogolo tsicana project for the knowledge acquired',
 'change from early marriage',
 'just thank them and ask that they continue to reinforce their services',
 'i

In [44]:
df_long = df[~df["very_short"]].copy()

df_changes = df_long[df_long["text_type"] == "changes_seen"]
df_recs = df_long[df_long["text_type"] == "recommendations"]

df_changes.shape, df_recs.shape

((928, 8), (946, 8))

In [45]:
df_recs.to_excel("/Users/anamargarida/Local/Coding/project/data_raw/recommendations_full.xlsx", index=False)

In [46]:
df_changes_sample = df_changes.sample(n=150, random_state=42)
df_recs_sample = df_recs.sample(n=150, random_state=42)

In [47]:
df_changes_sample.to_excel(
    "/Users/anamargarida/Local/Coding/project/data_clean/pilot_changes.xlsx",
    index=False
)

df_recs_sample.to_excel(
    "/Users/anamargarida/Local/Coding/project/data_clean/pilot_recommendations.xlsx",
    index=False
)

In [ ]:
path = "/Users/anamargarida/Local/Coding/project/data_clean/pilot_recommendations.xlsx"

df = pd.read_excel(path)

label_cols = [
    "continue_or_scale_project",
    "increase_information_awareness",
    "improve_service_access_quality",
    "delay_marriage_pregnancy",
    "support_girls_and_young_mothers",
    "community_engagement",
    "material_or_financial_support",
    "no_recommendation",
    "unspecified_positive_recommendation",
    "engaging_partners_in_srhr",
    "contraception_access_use_knowledge",
]

for col in label_cols:
    if col not in df.columns:
        df[col] = 0  # initialize as 0

if "coding_notes" not in df.columns:
    df["coding_notes"] = ""

# Save a new version so you don’t overwrite accidentally
out_path = "/Users/anamargarida/Local/Coding/project/data_clean/pilot_recommendations_coding.xlsx"
df.to_excel(out_path, index=False)

out_path

'/Users/anamargarida/Local/Coding/project/data_clean/pilot_recommendations_coding.xlsx'

In [49]:
# after manual coding:

path = "/Users/anamargarida/Local/Coding/project/data_clean/man_cod_recommendations.xlsx"
df = pd.read_excel(path)

label_cols = [
    "continue_or_scale_project",
    "increase_information_awareness",
    "improve_service_access",
    "support_girls_and_young_mothers",
    "community_engagement",
    "material_or_financial_support",
    "improve_staff_attitude_or_quality",
    "no_recommendation",
    "unspecified_positive_recommendation",
]

df[label_cols].sum().sort_values(ascending=False)

increase_information_awareness         39
support_girls_and_young_mothers        39
continue_or_scale_project              30
improve_service_access                 30
community_engagement                   22
no_recommendation                      17
unspecified_positive_recommendation    12
improve_staff_attitude_or_quality       7
material_or_financial_support           1
dtype: int64

In [50]:
df.shape

(150, 17)

In [36]:
# 1. Load both datasets
df_full = pd.read_excel("/Users/anamargarida/Local/Coding/project/data_raw/recommendations_full.xlsx")
df_pilot = pd.read_excel("/Users/anamargarida/Local/Coding/project/final/pilot_recommendations_coding_v1.xlsx")

In [ ]:
# 1) Force ID to the same type in BOTH
df_full["id"]  = pd.to_numeric(df_full["id"], errors="coerce").astype("Int64")
df_pilot["id"] = pd.to_numeric(df_pilot["id"], errors="coerce").astype("Int64")

# Optional but recommended: drop rows with missing IDs
df_full  = df_full.dropna(subset=["id"]).copy()
df_pilot = df_pilot.dropna(subset=["id"]).copy()

# 2) Define label cols from pilot (binary columns)
meta_cols = ["id","district","community","participant_profile","text_type","text","source"]
label_cols = [c for c in df_pilot.columns if c not in meta_cols]

# 3) Ensure full has those label columns (create if missing)
for c in label_cols:
    if c not in df_full.columns:
        df_full[c] = 0

# 4) Update full with pilot labels (only where IDs match)
df_full = df_full.set_index("id")
df_pilot = df_pilot.set_index("id")

common_ids = df_full.index.intersection(df_pilot.index)
missing_ids = df_pilot.index.difference(df_full.index)

df_full.loc[common_ids, label_cols] = df_pilot.loc[common_ids, label_cols]

df_full = df_full.reset_index()


# append the two missing pilot rows (so you keep all 150 labeled)
if len(missing_ids) > 0:
    df_missing = df_pilot.loc[missing_ids].reset_index()

    # make sure appended rows have every column df_full has
    for c in df_full.columns:
        if c not in df_missing.columns:
            df_missing[c] = np.nan

    df_missing = df_missing[df_full.columns]
    df_full = pd.concat([df_full, df_missing], ignore_index=True)

print("Pilot rows:", len(df_pilot))
print("Common IDs:", len(common_ids))
print("Missing IDs:", len(missing_ids), list(missing_ids)[:20])
print("Full rows:", len(df_full))
print("Duplicate IDs in full:", df_full["id"].duplicated().sum())
print("Pilot IDs present in full (count):", df_full["id"].isin(df_pilot.reset_index()["id"]).sum())


df_full.to_excel("/Users/anamargarida/Local/Coding/project/data_clean/Full_dataset_coded.xlsx", index=False)

Pilot rows: 150
Common IDs: 148
Missing IDs: 2 [np.int64(603039632), np.int64(604118792)]
Full rows: 946
Duplicate IDs in full: 0
Pilot IDs present in full (count): 148
